In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('../DataOut/bus_trip_all_points.csv')
# import bus trips
bus_trips = pd.read_csv("../DataOut/bus_trips.csv")

## Detect and remove Anomaly Trips

In [ ]:

from sklearn.metrics.pairwise import haversine_distances
from math import radians

# Define a function to calculate Haversine distance between two points
def haversine_distance(coord1, coord2):
    coord1 = [radians(x) for x in coord1]
    coord2 = [radians(x) for x in coord2]
    dist = haversine_distances([coord1, coord2])
    return dist[0][1] * 6371000.0  # Multiply by Earth radius to get distance in meters

# Define the anomaly detection function
def detect_anomalies(data, trip_id_column, longitude_column, latitude_column, reference_data, reference_trip_id, threshold_distance_meters=200):
    reference_locations = reference_data[reference_data[trip_id_column] == reference_trip_id]
    reference_locations = reference_locations[[latitude_column, longitude_column]].values
    
    anomalies = []
    for trip_id in data[trip_id_column].unique():
        trip_data = data[data[trip_id_column] == trip_id]
        gps_locations = trip_data[[latitude_column, longitude_column]].values
        
        anomaly_count = 0
        for location in gps_locations:
            min_distance = min([haversine_distance(location, ref_loc) for ref_loc in reference_locations])
            if min_distance > threshold_distance_meters:
                anomaly_count += 1
        print(f"anomaly count : {anomaly_count}")
        if anomaly_count / len(gps_locations) >= 0.1:
            anomalies.append(trip_id)
            print(f"anomaly trip id : {trip_id}")
    
    return anomalies

# Columns for trip ID, longitude, and latitude in your DataFrame
trip_id_column = 'trip_id'
longitude_column = 'longitude'
latitude_column = 'latitude'

# Reference data for correct GPS locations (longitude and latitude values)
reference_trip_id = 4  # Specify the trip ID for which to get the reference data
reference_data = data[data[trip_id_column] == reference_trip_id]  # Filter reference data

# Detect and remove anomaly trips
anomaly_trip_ids = detect_anomalies(data, trip_id_column, longitude_column, latitude_column, reference_data, reference_trip_id)
cleaned_data = data[~data[trip_id_column].isin(anomaly_trip_ids)]

print(f"Anomaly trips detected and removed: {anomaly_trip_ids}")

# save anomally trip id list in to csv file
anomaly_trip_ids_df = pd.DataFrame(anomaly_trip_ids)
anomaly_trip_ids_df.to_csv('../DataOut/anomaly_trip_ids.csv', index=False)



In [ ]:
# # # Anomaly trips detected and removed: [166.0, 193.0, 195.0, 207.0, 235.0, 1081.0, 1105.0, 141.0, 147.0, 153.0, 155.0, 157.0, 160.0, 171.0, 177.0, 190.0, 194.0, 206.0, 210.0, 226.0, 879.0]
# # # Anomaly trips detected and removed: [524.0, 851.0, 895.0, 1201.0, 1219.0, 1291.0, 1341.0, 1612.0, 1705.0, 1853.0, 1871.0, 1887.0, 1890.0, 1957.0, 1983.0, 1990.0, 1995.0, 1998.0, 2014.0, 2023.0, 2028.0, 2029.0, 2034.0, 2035.0, 2055.0, 2062.0, 2101.0, 2108.0, 2115.0, 2122.0, 2149.0, 2150.0, 2157.0, 2162.0, 2165.0, 2171.0, 2214.0, 2237.0, 2244.0, 2263.0, 2270.0, 2277.0, 2306.0, 2313.0, 2320.0, 2327.0, 2362.0, 2440.0, 2467.0, 2469.0, 2481.0, 2511.0, 2517.0, 2525.0, 2531.0, 2535.0, 2559.0, 2597.0, 2625.0, 2659.0, 2685.0, 2760.0, 2800.0, 2870.0, 2920.0, 3059.0, 3094.0, 3122.0, 3136.0, 3206.0, 3246.0, 3311.0, 3339.0, 3357.0, 3463.0, 3682.0, 4189.0, 4457.0, 4562.0, 4564.0, 4618.0, 4866.0, 5324.0, 5368.0, 5561.0, 5583.0, 120.0, 1778.0, 1982.0, 2019.0, 2036.0, 2052.0, 2415.0, 2421.0, 2427.0, 2429.0, 2431.0, 2434.0, 2445.0, 2451.0, 2464.0, 2468.0, 2480.0, 2484.0, 2500.0, 2514.0, 2524.0, 2528.0, 2532.0, 2536.0, 2550.0, 2558.0, 2566.0, 2568.0, 2574.0, 2576.0, 2588.0, 2594.0, 2598.0, 2600.0, 2610.0, 2616.0, 2624.0, 2640.0, 2648.0, 2676.0, 2684.0, 2692.0, 2702.0, 2710.0, 2718.0, 2720.0, 2732.0, 2733.0, 2747.0, 2755.0, 2763.0, 2765.0, 2767.0, 2781.0, 2795.0, 2803.0, 2851.0, 2867.0, 2921.0, 2935.0, 2943.0, 2977.0, 3245.0, 3310.0, 3338.0, 3356.0, 3456.0, 3669.0, 3787.0, 3970.0, 4188.0, 4304.0, 4561.0, 4663.0, 4763.0, 5043.0, 5185.0, 5519.0, 5560.0, 5573.0]
# # anomaly_trip_ids=[166.0, 193.0, 195.0, 207.0, 235.0, 1081.0, 1105.0, 141.0, 147.0, 153.0, 155.0, 157.0, 160.0, 171.0, 177.0, 190.0, 194.0, 206.0, 210.0, 226.0, 879.0]
# # anomaly_trip_ids=[524.0, 851.0, 895.0, 1201.0, 1219.0, 1291.0, 1341.0, 1612.0, 1705.0, 1853.0, 1871.0, 1887.0, 1890.0, 1957.0, 1983.0, 1990.0, 1995.0, 1998.0, 2014.0, 2023.0, 2028.0, 2029.0, 2034.0, 2035.0, 2055.0, 2062.0, 2101.0, 2108.0, 2115.0, 2122.0, 2149.0, 2150.0, 2157.0, 2162.0, 2165.0, 2171.0, 2214.0, 2237.0, 2244.0, 2263.0, 2270.0, 2277.0, 2306.0, 2313.0, 2320.0, 2327.0, 2362.0, 2440.0, 2467.0, 2469.0, 2481.0, 2511.0, 2517.0, 2525.0, 2531.0, 2535.0, 2559.0, 2597.0, 2625.0, 2659.0, 2685.0, 2760.0, 2800.0, 2870.0, 2920.0, 3059.0, 3094.0, 3122.0, 3136.0, 3206.0, 3246.0, 3311.0, 3339.0, 3357.0, 3463.0, 3682.0, 4189.0, 4457.0, 4562.0, 4564.0, 4618.0, 4866.0, 5324.0, 5368.0, 5561.0, 5583.0, 120.0, 1778.0, 1982.0, 2019.0, 2036.0, 2052.0, 2415.0, 2421.0, 2427.0, 2429.0, 2431.0, 2434.0, 2445.0, 2451.0, 2464.0, 2468.0, 2480.0, 2484.0, 2500.0, 2514.0, 2524.0, 2528.0, 2532.0, 2536.0, 2550.0, 2558.0, 2566.0, 2568.0, 2574.0, 2576.0, 2588.0, 2594.0, 2598.0, 2600.0, 2610.0, 2616.0, 2624.0, 2640.0, 2648.0, 2676.0, 2684.0, 2692.0, 2702.0, 2710.0, 2718.0, 2720.0, 2732.0, 2733.0, 2747.0, 2755.0, 2763.0, 2765.0, 2767.0, 2781.0, 2795.0, 2803.0, 2851.0, 2867.0, 2921.0, 2935.0, 2943.0, 2977.0, 3245.0, 3310.0, 3338.0, 3356.0, 3456.0, 3669.0, 3787.0, 3970.0, 4188.0, 4304.0, 4561.0, 4663.0, 4763.0, 5043.0, 5185.0, 5519.0, 5560.0, 5573.0]

# anom=pd.read_csv('../DataOut/anomaly_trip_ids.csv')
# anomaly_trip_ids = anom['0'].tolist()

# cleaned_data = data[~data['trip_id'].isin(anomaly_trip_ids)]

# data= cleaned_data

# print(f"Anomaly trips detected and removed: {anomaly_trip_ids}")

## feature Eng

### Add Acceleration

In [ ]:
import pandas as pd
import numpy as np

# Assuming you have a DataFrame named 'data' with columns: 'speed', 'devicetime', and 'trip_id'
# Convert the 'devicetime' column to pandas Timestamp if it's not already
data['devicetime'] = pd.to_datetime(data['devicetime'])

# Calculate time difference between consecutive rows within the same trip
data['time_diff'] = data.groupby('trip_id')['devicetime'].diff()

# Calculate change in speed between consecutive rows within the same trip
data['speed_diff'] = data.groupby('trip_id')['speed'].diff()

# Filter out rows with time differences close to zero
data = data[data['time_diff'].dt.total_seconds() > 0.001]  # Adjust the threshold as needed

# Calculate acceleration by dividing speed difference by time difference (avoiding division by zero)
data['acceleration'] = data['speed_diff'] / data['time_diff'].dt.total_seconds()

# Handle the case where time difference is zero or very close to zero, set acceleration to 0
data['acceleration'].replace([np.inf, -np.inf], np.nan, inplace=True)
data['acceleration'].fillna(0, inplace=True)

print(data)


In [ ]:
# fill Nan values with 0
data['acceleration'].fillna(0, inplace=True)
# drop time_diff and speed_diff columns
data.drop(['time_diff', 'speed_diff'], axis=1, inplace=True)

In [ ]:
# save to csv
data.to_csv('../DataOut/bus_trip_all_points_with_acceleration.csv', index=False)

### Radial Acc

In [ ]:
from math import cos, radians, sqrt
from pyproj import Proj, Transformer

def lat_lon_to_utm(lat, lon):
    # Create a UTM projection for the appropriate UTM zone (Zone 44N for Sri Lanka)
    utm_zone = 44
    utm_proj = Proj(proj='utm', zone=utm_zone, ellps='WGS84')

    # Convert latitude and longitude to UTM coordinates
    utm_easting, utm_northing = utm_proj(lon, lat)
    return utm_easting, utm_northing
# Calculate radius of curvature using OpenStreetMap API

def calculate_radius_of_curvature(x1, y1, x2, y2, x3,y3):
    x12 = x1 - x2;
    x13 = x1 - x3;
 
    y12 = y1 - y2;
    y13 = y1 - y3;
 
    y31 = y3 - y1;
    y21 = y2 - y1;
 
    x31 = x3 - x1;
    x21 = x2 - x1;
 
    # x1^2 - x3^2
    sx13 = pow(x1, 2) - pow(x3, 2);
 
    # y1^2 - y3^2
    sy13 = pow(y1, 2) - pow(y3, 2);
 
    sx21 = pow(x2, 2) - pow(x1, 2);
    sy21 = pow(y2, 2) - pow(y1, 2);
    
    denominator = 2 * ((y31) * (x12) - (y21) * (x13))
    
    # Avoid division by zero
    if denominator == 0:
        return 0  # Or some other value you want to use
        
    f = (((sx13) * (x12) + (sy13) *
          (x12) + (sx21) * (x13) +
          (sy21) * (x13)) // denominator)
             
    g = (((sx13) * (y12) + (sy13) * (y12) +
          (sx21) * (y13) + (sy21) * (y13)) //
          (2 * ((x31) * (y12) - (x21) * (y13))));
 
    c = (-pow(x1, 2) - pow(y1, 2) -
         2 * g * x1 - 2 * f * y1);
 
    # eqn of circle be x^2 + y^2 + 2*g*x + 2*f*y + c = 0
    # where centre is (h = -g, k = -f) and
    # radius r as r^2 = h^2 + k^2 - c
    h = -g;
    k = -f;
    sqr_of_r = h * h + k * k - c;
 
    # r is the radius
    r = round(sqrt(sqr_of_r), 5);
    return r
    

# Calculate radial acceleration based on speed and radius of curvature
def calculate_radial_acceleration(speed, radius_of_curvature):
    if radius_of_curvature != 0:
        radial_acceleration = speed**2 / radius_of_curvature
    else:
        radial_acceleration = 0
    return radial_acceleration

# Create a dictionary to store DataFrames for each trip ID
trip_dataframes = {}
window_size = 1  # Number of rows to use for calculating radius of curvature
trips=data['trip_id'].unique()
# Iterate over trip IDs to calculate radial acceleration for each trip's DataFrame
for trip_id in trips:
    trip_df = data[data["trip_id"] == trip_id].copy()  # Get DataFrame for the current trip
    
    # Reset the index of trip_df only once
    trip_df.reset_index(drop=True, inplace=True)
    
    for i in range(window_size, len(trip_df) - window_size):
        lat1, lon1 = trip_df.iloc[i - window_size]["latitude"], trip_df.iloc[i - window_size]["longitude"]
        lat2, lon2 = trip_df.iloc[i]["latitude"], trip_df.iloc[i]["longitude"]
        lat3, lon3 = trip_df.iloc[i + window_size]["latitude"], trip_df.iloc[i + window_size]["longitude"]
        
        x1, y1 = lat_lon_to_utm(lat1, lon1)
        x2, y2 = lat_lon_to_utm(lat2, lon2)
        x3, y3 = lat_lon_to_utm(lat3, lon3)

        radius_of_curvature = calculate_radius_of_curvature(x1, y1, x2, y2, x3, y3)
        speed = trip_df.iloc[i]["speed"]

        radial_acceleration = calculate_radial_acceleration(speed, radius_of_curvature)
        trip_df.at[i, "radial_acceleration"] = radial_acceleration

    trip_dataframes[trip_id] = trip_df  # Store the calculated DataFrame for the trip

# Concatenate all trip-specific dataframes into one
concatenated_df = pd.concat(trip_dataframes.values(), ignore_index=True)

# Iterate through rows and assign radial acceleration values to the original dataframe
for index, row in concatenated_df.iterrows():
    data.loc[data['id'] == row['id'], 'radial_acceleration'] = row['radial_acceleration']

print(data)

In [ ]:
# fill Nan values with 0
data['radial_acceleration'].fillna(0, inplace=True)

In [ ]:
data.to_csv('../DataOut/bus_trip_all_points_with_acceleration_and_radial_acceleration.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming you have 'data' DataFrame containing bus data

# Convert 'devicetime' column to datetime
data['devicetime'] = pd.to_datetime(data['devicetime'])

# Draw a line graph of radial acceleration against time for a single trip
def draw_radial_acceleration_graph(trip_id):
    trip_df = data[data["trip_id"] == trip_id]
    plt.figure(figsize=(10, 6))  # Adjust figure size as needed
    plt.plot(trip_df["devicetime"], trip_df["radial_acceleration"])
    plt.xlabel("Time")
    plt.ylabel("radial_acceleration")
    plt.title("Trip " + str(trip_id))
    plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
    plt.tight_layout()  # Adjust layout for better formatting
    plt.show()

draw_radial_acceleration_graph(4)
draw_radial_acceleration_graph(2)
draw_radial_acceleration_graph(20)


### Distance from start

#### Cumalative addition of distance

In [ ]:
import pandas as pd
import numpy as np

# Assuming you have a DataFrame named 'data' with columns: 'trip_id', 'latitude', 'longitude', and 'devicetime'
# Convert the 'devicetime' column to pandas Timestamp if it's not already
data['devicetime'] = pd.to_datetime(data['devicetime'])

# Function to calculate Haversine distance between two points
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth's radius in meters
    phi_1 = np.radians(lat1)
    phi_2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2.0)**2 + np.cos(phi_1) * np.cos(phi_2) * np.sin(delta_lambda / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    meters = R * c
    return meters

# Calculate the distance between consecutive rows within the same trip
data['distance'] = data.groupby('trip_id').apply(lambda group: haversine_distance(group['latitude'], group['longitude'], group['latitude'].shift(), group['longitude'].shift())).reset_index(level=0, drop=True)

# Calculate cumulative distance for each trip
data['cumulative_distance'] = data.groupby('trip_id')['distance'].cumsum()

print(data)


In [ ]:
# drop distance column and rename cumulative_distance to distance from start
data.drop(['distance'], axis=1, inplace=True)
data.rename(columns={'cumulative_distance': 'distance_from_start'}, inplace=True)


In [ ]:
# fill nan with 0
data['distance_from_start'].fillna(0, inplace=True)

In [ ]:
# save as csv file 
data.to_csv('../DataOut/bus_trip_all_points_with_acceleration_and_radial_acceleration_and_distance.csv', index=False)

## Get Accelaration and Breaks

In [ ]:
import pandas as pd
import numpy as np

# Assuming you have a DataFrame named 'data' with columns: 'acceleration', 'devicetime', and 'trip_id'
# Convert the 'devicetime' column to pandas Timestamp if it's not already
data['devicetime'] = pd.to_datetime(data['devicetime'])

# Calculate time difference between consecutive rows within the same trip
data['time_diff'] = data.groupby('trip_id')['devicetime'].diff()

# Calculate change in acceleration between consecutive rows within the same trip
data['acc_diff'] = data.groupby('trip_id')['acceleration'].diff()

# Filter out rows with time differences close to zero
data = data[data['time_diff'].dt.total_seconds() > 0.001]  # Adjust the threshold as needed

# Calculate acceleration by dividing acceleration difference by time difference (avoiding division by zero)
data['acceleration_der'] = data['acc_diff'] / data['time_diff'].dt.total_seconds()

# Handle the case where time difference is zero or very close to zero, set acceleration_der to 0
data['acceleration_der'].replace([np.inf, -np.inf], np.nan, inplace=True)
data['acceleration_der'].fillna(0, inplace=True)

data


In [ ]:
# save as csv file 
data.to_csv('../DataOut/bus_trip_all_points_with_acceleration_and_radial_acceleration_and_distance_and_accDiff.csv', index=False)

In [ ]:
# Draw a line graph of radial acceleration against time for a single trip
def drawAgainst(data,trip_id,columnx,columny):
    trip_df = data[data["trip_id"] == trip_id]
    plt.figure(figsize=(10, 6))  # Adjust figure size as needed
    plt.plot(trip_df[f"{columnx}"], trip_df[f"{columny}"])
    plt.xlabel(f"{columnx}")
    plt.ylabel(f"{columny}")
    plt.title("Trip " + str(trip_id))
    plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
    plt.tight_layout()  # Adjust layout for better formatting
    plt.show()


In [ ]:
drawAgainst(data,116,'distance_from_start','acceleration_der')
drawAgainst(data,1365,'distance_from_start','acceleration_der')
drawAgainst(data,4,'distance_from_start','acceleration_der')

## Check whether the time or distance is better

In [ ]:
drawAgainst(data,193,'distance_from_start','radial_acceleration')
drawAgainst(data,5,'distance_from_start','radial_acceleration')
drawAgainst(data,10,'distance_from_start','radial_acceleration')

print("=========================================================")

drawAgainst(data,4,'devicetime','radial_acceleration')
drawAgainst(data,5,'devicetime','radial_acceleration')
drawAgainst(data,10,'devicetime','radial_acceleration')

Seem like distance is better